# Volume-Synchronized Probability of Informed Trading (VPIN)

$\text{VPIN}_\tau = \frac{|V_\tau^\text{buy} - V_\tau^\text{sell}|}{V_\tau^\text{buy} + V_\tau^\text{sell}}$, where $\tau$ is the 10-minute interval.

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [136]:
# import packages
import numpy as np
import pandas as pd
import sys

# Paths tricks
import os
from pathlib import Path

In [137]:
# some functions
def select_stock_book(book_path,stock_id,time_id=-1):
    
    stock_path = os.path.join(book_path, 'stock_id=' + str(stock_id))
    book = pd.read_parquet(stock_path)
    
    if time_id != -1: 
        book = book[book['time_id'] == time_id]
        
    return book

def select_stock_trades(trade_path,stock_id,time_id=-1):
    
    trade_path = os.path.join(trade_path, 'stock_id=' + str(stock_id))
    trades = pd.read_parquet(trade_path)
    
    if time_id != -1: 
        trades = trades[trades['time_id'] == time_id]
        
    return trades

In [138]:
# load the data
datapath = os.path.join(str(Path.home()), 'ownCloud', 'Data', 'Kaggle', 'optiver-realized-volatility-prediction')

book = select_stock_book(book_path=os.path.join(datapath, 'book_train.parquet'), stock_id=0, time_id=5)
trades = select_stock_trades(trade_path=os.path.join(datapath, 'trade_train.parquet'), stock_id=0, time_id=5)

# data manipulation
trades['avg_size_per_order'] = trades['size']/trades['order_count'] # add average trade size
trades['prev_trade_second'] = trades['seconds_in_bucket'].shift()

# add time length for each state of the book
book['time_length'] = book['seconds_in_bucket'].diff().shift(periods=-1)
book.loc[len(book)-1, 'time_length'] = 600 - book['seconds_in_bucket'].iloc[-1]

In [139]:
trades.head()

,time_id,seconds_in_bucket,price,size,order_count,avg_size_per_order,prev_trade_second
0,5,21,1.002301,326,12,27.166667,NaN
1,5,46,1.002778,128,4,32.000000,21.0
2,5,50,1.002818,55,1,55.000000,46.0
3,5,57,1.003155,121,5,24.200000,50.0
4,5,68,1.003646,4,1,4.000000,57.0


In [140]:
# a function to compute buyer initiated trades probability
def buyer_prob(price, bid, ask):
    return max(0, min(1, (price - bid)/(ask - bid)))

def signed_volume(df_book, start_time, end_time, price, volume, output='buy'):
    if output not in ['buy', 'sell']:
        sys.exit("Required output = 'buy' or 'sell'")
    
    if np.isnan(start_time):
        start_time = 0
    
    # compute the weighted bid and ask prices from the book
    w_ask = np.sum(df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['ask_price1'] * df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['time_length']) / (end_time - start_time)
    w_bid = np.sum(df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['bid_price1'] * df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['time_length']) / (end_time - start_time)
    
    if output == 'buy':
        result = volume * buyer_prob(price, w_bid, w_ask)
    else:
        result = volume * (1 - buyer_prob(price, w_bid, w_ask))

    return result

In [141]:
trades['buyer_volume'] = trades.apply(lambda row: signed_volume(book, row['prev_trade_second'], row['seconds_in_bucket'], row['price'], row['size']), axis=1)
trades['seller_volume'] = trades['size'] - trades['buyer_volume']
trades['vpin'] = trades.apply(lambda row: np.abs(row['buyer_volume'] - row['seller_volume'])/row['size'], axis = 1)
trades.head()

,time_id,seconds_in_bucket,price,size,order_count,avg_size_per_order,prev_trade_second,buyer_volume,seller_volume,vpin
0,5,21,1.002301,326,12,27.166667,NaN,325.960008,0.039992,0.999755
1,5,46,1.002778,128,4,32.000000,21.0,123.438110,4.561890,0.928720
2,5,50,1.002818,55,1,55.000000,46.0,0.000000,55.000000,1.000000
3,5,57,1.003155,121,5,24.200000,50.0,121.000000,0.000000,1.000000
4,5,68,1.003646,4,1,4.000000,57.0,4.000000,0.000000,1.000000


In [142]:
# calculating VPIN for time_id by weighting by size
VPIN = np.sum(trades['vpin'] * trades['size']) / np.sum(trades['size'])
print(VPIN)

0.9234727992343469


## check with using the support_file

In [2]:
# import packages
import numpy as np
import pandas as pd
import sys

# Paths tricks
import os
from pathlib import Path

from support_file import *

In [3]:
# load the data
datapath = os.path.join(str(Path.home()), 'ownCloud', 'Data', 'Kaggle', 'optiver-realized-volatility-prediction')

book_0 = pd.read_parquet(os.path.join(datapath, 'book_train.parquet', 'stock_id=0'))
trades_0 = pd.read_parquet(os.path.join(datapath, 'trade_train.parquet', 'stock_id=0'))

In [4]:
vpin_per_time_id(book_0[book_0.time_id==11], trades_0[trades_0.time_id==11])

/Users/vladimir.levin/Yandex.Disk.localized/01_Documents/01_Study/0_LUX/01_PhD/Drafts/Project3/kaggle_volatility/support_file.py:1675: RuntimeWarning: invalid value encountered in true_divide
  w_ask = np.sum(df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['ask_price1'] * df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['time_length']) / (end_time - start_time)
/Users/vladimir.levin/Yandex.Disk.localized/01_Documents/01_Study/0_LUX/01_PhD/Drafts/Project3/kaggle_volatility/support_file.py:1676: RuntimeWarning: invalid value encountered in true_divide
  w_bid = np.sum(df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['bid_price1'] * df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['time_length']) / (end_time - start_time)


0.8251119007335302

In [17]:
trades_0['time_id'].unique()[0:5]

array([ 5, 11, 16, 31, 62], dtype=int16)

In [61]:
import time

In [62]:
program_start = time.time()
vpin_table = pd.DataFrame({'time_id':trades_0['time_id'].unique(), 'vpin':-1}) #initial table to fill

# this loop takes long time (7.3 min)
for i in  trades_0['time_id'].unique():
    vpin_value = vpin_per_time_id(book_0[book_0.time_id==i], trades_0[trades_0.time_id==i]) # calculate vpin for this time_id
    vpin_table.at[vpin_table.index[vpin_table.time_id == i].tolist(), 'vpin'] = vpin_value # update the table at the required row
    print(i)
    
time_now = time.time()
print(f'Total time for the loop = {time_now - program_start}')
vpin_table
    

5
11
16


/Users/vladimir.levin/Yandex.Disk.localized/01_Documents/01_Study/0_LUX/01_PhD/Drafts/Project3/kaggle_volatility/support_file.py:1675: RuntimeWarning: invalid value encountered in true_divide
  w_ask = np.sum(df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['ask_price1'] * df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['time_length']) / (end_time - start_time)
/Users/vladimir.levin/Yandex.Disk.localized/01_Documents/01_Study/0_LUX/01_PhD/Drafts/Project3/kaggle_volatility/support_file.py:1676: RuntimeWarning: invalid value encountered in true_divide
  w_bid = np.sum(df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['bid_price1'] * df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['time_length']) / (end_time - start_time)


31
62
72
97
103
109

/Users/vladimir.levin/Yandex.Disk.localized/01_Documents/01_Study/0_LUX/01_PhD/Drafts/Project3/kaggle_volatility/support_file.py:1675: RuntimeWarning: invalid value encountered in true_divide
  w_ask = np.sum(df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['ask_price1'] * df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['time_length']) / (end_time - start_time)
/Users/vladimir.levin/Yandex.Disk.localized/01_Documents/01_Study/0_LUX/01_PhD/Drafts/Project3/kaggle_volatility/support_file.py:1676: RuntimeWarning: invalid value encountered in true_divide
  w_bid = np.sum(df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['bid_price1'] * df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['time_length']) / (end_time - start_time)



123
128
146
147


...

32753


/Users/vladimir.levin/Yandex.Disk.localized/01_Documents/01_Study/0_LUX/01_PhD/Drafts/Project3/kaggle_volatility/support_file.py:1675: RuntimeWarning: invalid value encountered in true_divide
  w_ask = np.sum(df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['ask_price1'] * df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['time_length']) / (end_time - start_time)
/Users/vladimir.levin/Yandex.Disk.localized/01_Documents/01_Study/0_LUX/01_PhD/Drafts/Project3/kaggle_volatility/support_file.py:1676: RuntimeWarning: invalid value encountered in true_divide
  w_bid = np.sum(df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['bid_price1'] * df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['time_length']) / (end_time - start_time)


32758
32763
32767
Total time for the loop = 439.5459682941437


/Users/vladimir.levin/Yandex.Disk.localized/01_Documents/01_Study/0_LUX/01_PhD/Drafts/Project3/kaggle_volatility/support_file.py:1675: RuntimeWarning: invalid value encountered in true_divide
  w_ask = np.sum(df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['ask_price1'] * df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['time_length']) / (end_time - start_time)
/Users/vladimir.levin/Yandex.Disk.localized/01_Documents/01_Study/0_LUX/01_PhD/Drafts/Project3/kaggle_volatility/support_file.py:1676: RuntimeWarning: invalid value encountered in true_divide
  w_bid = np.sum(df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['bid_price1'] * df_book[(start_time <= df_book['seconds_in_bucket']) & (df_book['seconds_in_bucket'] < end_time)]['time_length']) / (end_time - start_time)


,time_id,vpin
0,5,0.923473
1,11,0.825112
2,16,0.994899
3,31,0.961566
4,62,0.924492
...,...,...
3825,32751,0.834700
3826,32753,0.903424
3827,32758,0.964881
3828,32763,0.896550


In [64]:
vpin_table.describe()

,time_id,vpin
count,3830.000000,3830.000000
mean,16038.933681,0.916824
std,9366.261949,0.067022
min,5.000000,0.433877
25%,7856.500000,0.888682
50%,15852.500000,0.933161
75%,23988.500000,0.964354
max,32767.000000,1.000000


In [65]:
vpin_table.head()

,time_id,vpin
0,5,0.923473
1,11,0.825112
2,16,0.994899
3,31,0.961566
4,62,0.924492


In [5]:
df_fin_metrics_trades = trades_0.groupby(['time_id']).apply(fin_metrics_trades_data).to_frame().reset_index()

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/nanops.py:1516: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [7]:
df_fin_metrics_trades = df_fin_metrics_trades.rename(columns={0:'embedding'})
df_fin_metrics_trades

,time_id,embedding
0,5,"[110, 3190.139181494713, 28.9, 0.0002036144192..."
1,11,"[57, 1289.3534323573112, 22.614035087719298, 0..."
2,16,"[68, 2158.608928322792, 31.779411764705884, 0...."
3,31,"[59, 1959.60554677248, 33.25423728813559, 0.00..."
4,62,"[89, 1790.2544958591461, 20.123595505617978, 6..."
...,...,...
3825,32751,"[159, 3441.815545618534, 21.69811320754717, 0...."
3826,32753,"[109, 4548.671492576599, 41.71559633027523, 0...."
3827,32758,"[126, 4247.563002169132, 33.73015873015873, 0...."
3828,32763,"[114, 3224.4217962026596, 28.219298245614034, ..."


In [8]:
list_vlad_trades = []

In [14]:
# df_fin_metrics_trades[['trades_number', 'traded_volume', 'avg_trade_size', 'roll_measure', 'roll_impact', 'mkt_impact', 'amihud']] = pd.DataFrame(df_fin_metrics_trades.embedding.tolist(), index=df_fin_metrics_trades.index)
# stock_id = 0
# df_fin_metrics_trades['time_id'] = [f'{stock_id}-{time_id}' for time_id in df_fin_metrics_trades['time_id']] 
# df_fin_metrics_trades = df_fin_metrics_trades.rename(columns={'time_id':'row_id'}).drop(['embedding'],axis=1)

# df_fin_metrics_trades

list_vlad_trades.append(df_fin_metrics_trades)
list_vlad_trades

[       row_id  trades_number  traded_volume  avg_trade_size  roll_measure  \
 0         0-5            110    3190.139181       28.900000      0.000204   
 1        0-11             57    1289.353432       22.614035      0.000206   
 2        0-16             68    2158.608928       31.779412      0.000457   
 3        0-31             59    1959.605547       33.254237      0.000516   
 4        0-62             89    1790.254496       20.123596      0.000065   
 ...       ...            ...            ...             ...           ...   
 3825  0-32751            159    3441.815546       21.698113      0.000207   
 3826  0-32753            109    4548.671493       41.715596      0.000239   
 3827  0-32758            126    4247.563002       33.730159      0.000248   
 3828  0-32763            114    3224.421796       28.219298      0.000186   
 3829  0-32767             70    3674.366695       52.557143      0.000022   
 
        roll_impact  mkt_impact        amihud  
 0     6.38261